In [2]:
!pip install -qU gradio python-dotenv langchain-upstage langchain langchain-chroma

In [3]:
%load_ext dotenv
%dotenv

In [17]:
import warnings

warnings.filterwarnings("ignore")

In [18]:
import gradio as gr

from langchain_upstage import ChatUpstage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import AIMessage, HumanMessage


llm = ChatUpstage()

In [19]:
from langchain.docstore.document import Document
text = ""
with open('gongja.txt', 'r') as f:
    for line in f:
        text += line.strip("\\xa0") + " "
text.strip()
print(text[:100])

≪  論  語  ≫
  
 <學而第一>
  
 <學而第一>01 子曰, “學而時習之, 不亦說乎? 有朋自遠方來, 不亦樂乎? 人不知而不慍, 不亦君子乎?”
 공자께서 말씀하셨다. “배우고


In [20]:
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

# 2. Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.create_documents([text])
print("Splits:", len(splits))

Splits: 114


In [21]:
from langchain_chroma import Chroma
from langchain_upstage import UpstageEmbeddings
from langchain.docstore.document import Document

vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=UpstageEmbeddings(model="solar-embedding-1-large"),
)

retriever = vectorstore.as_retriever()



In [68]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", """너는 공자의 말씀을 배운 챗봇이야.
         논어에 나오는 내용을 context로 줄테니 이를 참고해서 대답해줘.
         Context에 관련 내용이 있다면 Context의 내용을 참고해서 답변해줘. 그 구절을 답변에 이용해도 좋아
         Context에 관련된 내용이 없다면 Context에 있는 내용을 참고해서 공자가 대답할 수 있는 내용을 대답해줘.
         ---------------------------------------------------------------
         Context: {context}
         """),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{message}"),
    ]
)
chain = prompt_template | llm | StrOutputParser()

In [69]:
def chat(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    context = retriever.invoke(message)

    return chain.invoke({"message": message, "history": history_langchain_format, "context": context})

In [70]:
with gr.Blocks() as demo:
    chatbot = gr.ChatInterface(
        chat,
        examples=[
            "인이란 무엇입니까",
            "예란 무엇입니까",
            "인생이란 무엇입니까",
        ],
        title="Solar Chatbot",
        description="Upstage Solar Chatbot",
    )
    chatbot.chatbot.height = 300

In [71]:
if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7904

To create a public link, set `share=True` in `launch()`.
